In [ ]:
import os
import pandas as pd

path2 = 'Twitter_Scrape\Target_Harris\Anti\Comments'
path3 = 'Twitter_Scrape\Target_Trump\Anti\Comments'
path4 = 'Twitter_Scrape\Target_Trump\Pro\Comments'
# Define the folder path
folder_path = path2

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Process each row in the 'Text' column
        for index, row in df.iterrows():
            
            # Add your processing logic here
            df['mentions'] = df['comment_text'].apply(lambda x: ' '.join(re.findall(r'@\w+', x)))
            df['new_hashtags'] = df['comment_text'].apply(lambda x: ' '.join(re.findall(r'#\w+', x)))
            
            # Initialize the VADER sentiment analyzer
            nltk.download('vader_lexicon')
            sia = SentimentIntensityAnalyzer()

            # Perform sentiment analysis on the 'Text in English language' column and save the results to a new column
            def analyze_sentiment(text):
                sentiment_scores = sia.polarity_scores(text)
                return sentiment_scores['pos'], sentiment_scores['neu'], sentiment_scores['neg'], sentiment_scores['compound']

            # Apply the analyze_sentiment function to the 'Text in English language' column
            df['Positive'], df['Neutral'], df['Negative'], df['Compound'] = zip(*df['comment_text'].apply(analyze_sentiment))
            
            import spacy
            # Load the spaCy English model
            nlp = spacy.load("en_core_web_sm")

            # Function to perform NER and separate entities by type
            def extract_entities_by_type(text):
                doc = nlp(text)
                persons = [entity.text for entity in doc.ents if entity.label_ == "PERSON"]
                orgs = [entity.text for entity in doc.ents if entity.label_ == "ORG"]
                gpes = [entity.text for entity in doc.ents if entity.label_ == "GPE"]
                rel = [entity.text for entity in doc.ents if entity.label_ == "NORP"]
                dat = [entity.text for entity in doc.ents if entity.label_ == "DATE"]
                return persons, orgs, gpes, rel, dat

            # Apply the function to each tweet and store the results in separate columns
            df[['Persons', 'Organizations', 'Locations', 'Nationalities/religiious groups', 'Date']] = df['comment_text'].apply(
                lambda x: pd.Series(extract_entities_by_type(x)))

        
        # Save the processed dataframe to the same CSV file, overriding the old one
        df.to_csv(file_path, index=False)